<a href="https://colab.research.google.com/github/sm-aamer/Langchain/blob/main/Project_2_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [37]:
!pip install langchain_community pypdf langchain_google_genai langchain_chroma -q

In [38]:
!pip install  --upgrade -q google-auth google-auth-oauthlib google-auth-httplib2  google-cloud

In [39]:
from google.colab import userdata
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')

In [40]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("/content/PIAIC_Course_Catalog_v1_1.pdf")
data = loader.load()

In [41]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000)
docs = text_splitter.split_documents(data)


print("total number of documents : " ,len(docs))

total number of documents :  20


In [42]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/content/cedar-calling-447105-m5-4e0bf896ba0f.json"

In [43]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
embedding_model = GoogleGenerativeAIEmbeddings(model="models/embedding-001", google_api_key=GOOGLE_API_KEY)

In [51]:
vectorstore = Chroma.from_documents(documents=docs, embedding=embedding_model)
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 10})

In [52]:
from langchain_chroma import Chroma
vectorstores = Chroma(collection_name="rag-system", embedding_function=embedding_model)

In [53]:
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model = "gemini-2.0-flash-exp", temperature=0.3, max_token=1000)

In [54]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

In [55]:
user_promt = (
    "You are a chatbot assistant for addressing the user query and answering."
    "Which will use the following pieces of retrieved data and reply in an expert manner."
    "Moreover, you will use a maximum of three sentences and keep the answer concise."
    "If you don't know the answer to the user question,"
    "just say that you don't have sufficient information about it."
    "/n/n"
    "{context}"
)
prompt = ChatPromptTemplate.from_messages(
    [("system", user_promt), ("human", "{input}")]
)

In [56]:
question_answering_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answering_chain)

In [60]:
from IPython.display import Markdown
response = rag_chain.invoke({"input" : "What is PIAIC Philosophy"})
display(Markdown(response["answer"]))

PIAIC's philosophy includes holding classes in the evenings or on weekends, keeping fees low, supplementing onsite classes with online content, and offering online classes for those who cannot attend onsite. Admissions are based on entry tests for Math and English capabilities, and students must choose a technical track. Additionally, all students are required to take the innovation track.

In [59]:
from IPython.display import Markdown
response = rag_chain.invoke({"input" : "What programs are being offered?"})
display(Markdown(response["answer"]))

The PIAIC offers technical training programs in Artificial Intelligence, Cloud Native Computing, Internet of Things, Blockchain, Quantum Computing, and 5G, Network Programmability, and Software Defined Networking. Additionally, Computational Genomics and 3D Printing programs are under development. These programs are designed to prepare students for the global market.